# Creating a NeuralNet from scratch

  

### The structure of the project

The first order of business was defining a `Layer` class.

The `Dense` and `ActivationLayer` Classes would then inherit from that class.

Then afterwards, I implemented a `Network` class which will handle the training of the NeuralNet.


For the optimiser, I implemented `ADAM` which takes in the gradients obtained by `Network.train()` and then updates the parameters according to their first and second moments.

### Results and interpretations

I have trained two models, one on a regression task and the other on a  binary classification one to test them out.

* For the classification task, I chose the Wisconsin breast cancer dataset.

  The model then, needs to predict whether the tumor was benign or not.


  The model returned an accuracy of **96.9%** (Train) & **97.7%** (Test).



* For the regression task, I chose the California housing dataset.
  

  Based on the features found in the dataset, the model would then predict the housing prices.

  The model returned a *mean squared error* of **0.4071** meaning that, on average, the model's predictions were off by about \$4,071 (since the prices were in **$100,000**).

## Defining the Layer class

*A base class for layers with methods for forward and backward passes.*


In [12]:
import numpy as np

In [13]:
class Layer:
    def __init__(self):

        pass

    def forward(self, input_data):
        # Forward Pass

        return input_data

    def backward(self, output_gradient):
        # Backward Pass

        return output_gradient


## Implementing the dense layer

*A dense (fully connected) layer inheriting from the base Layer class.*


In [14]:
class Dense(Layer):

    def __init__(self, input_size, output_size):

        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(input_size, output_size) * 0.01  # weights initialised with small random values
        self.biases = np.zeros((1, output_size))  # biases initialised as 0
        self.input_data = None  # input for backward pass
        self.weights_gradient = None # weights gradient
        self.biases_gradient = None # biases gradient

    def forward(self, input_data):

        self.input_data = input_data
        return np.dot(input_data, self.weights) + self.biases

    def backward(self, output_gradient):

        # Calculate and store gradients
        self.weights_gradient = np.dot(self.input_data.T, output_gradient)
        self.biases_gradient = np.sum(output_gradient, axis=0, keepdims=True)
        input_gradient = np.dot(output_gradient, self.weights.T)

        return input_gradient

    def get_params_and_grads(self):

        return [
            {'params': self.weights, 'grads': self.weights_gradient},
            {'params': self.biases, 'grads': self.biases_gradient}
        ]



## Implementing activation functions

*Activation functions like ReLU, Sigmoid and, Tanh which would be used by the NN*


In [15]:
# Base Class for activation functions
class ActivationLayer(Layer):

    def __init__(self):

        pass

    def forward(self, input_data):

        return input_data

    def backward(self, output_gradient):

        return output_gradient


# Various activation functions which inherit from ActivationLayer class
class ReLU(ActivationLayer):

    def __init__(self):

        self.input_data = None

    def forward(self, input_data):

        self.input_data = input_data
        return np.maximum(0, input_data)

    def backward(self, output_gradient):

        relu_gradient = (self.input_data > 0).astype(float)
        return output_gradient * relu_gradient

class Sigmoid(ActivationLayer):

    def __init__(self):

        self.output_data = None

    def forward(self, input_data):

        output = 1 / (1 + np.exp(-input_data))
        self.output_data = output
        return output

    def backward(self, output_gradient):

        sigmoid_gradient = self.output_data * (1 - self.output_data)
        return output_gradient * sigmoid_gradient

class Tanh(ActivationLayer):


    def __init__(self):

        self.output_data = None

    def forward(self, input_data):

        output = np.tanh(input_data)
        self.output_data = output
        return output

    def backward(self, output_gradient):

        tanh_gradient = 1 - self.output_data**2
        return output_gradient * tanh_gradient


## Implementing the network class


*A class which representing the Neural net, allowing it to hold the layers and manage the training process*


In [27]:
class Network:


    def __init__(self):

        self.layers = []


    def add_layer(self, layer):

        self.layers.append(layer)

    def forward(self, input_data):

        output = input_data
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def backward(self, output_gradient):

        input_gradient = output_gradient
        for layer in reversed(self.layers):
            input_gradient = layer.backward(input_gradient)

    def get_trainable_params_and_grads(self):
        # Collects trainable parameters and their gradients from all layers
        params_and_grads = []
        for layer in self.layers:
            if hasattr(layer, 'get_params_and_grads'):
                params_and_grads.extend(layer.get_params_and_grads())
        return params_and_grads


    def train(self, x_train, y_train, loss_function, optimizer, epochs, batch_size):


        num_samples = x_train.shape[0]

        for epoch in range(epochs):
            epoch_loss = 0

            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            x_train_shuffled = x_train[indices]
            y_train_shuffled = y_train[indices]


            for i in range(0, num_samples, batch_size):
                x_batch = x_train_shuffled[i:i + batch_size]
                y_batch = y_train_shuffled[i:i + batch_size]

                # Forward pass
                output = self.forward(x_batch)

                # Calculate loss
                loss = loss_function.loss(y_batch, output)
                epoch_loss += loss

                # Backward pass
                output_gradient = loss_function.gradient(y_batch, output)
                self.backward(output_gradient)

                # Update weights
                params_and_grads = self.get_trainable_params_and_grads()
                optimizer.update(params_and_grads)


            avg_epoch_loss = epoch_loss / (num_samples / batch_size)


            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_epoch_loss}")

## Implementing a loss function

*A class for a loss function like Mean Squared Error or Cross-Entropy with methods for calculating the loss and its gradient*


In [17]:
class LossFunction:

    def loss(self, y_true, y_pred):
        # Calculations would be handled by the subclass
        raise NotImplementedError("Subclass must implement abstract method")

    def gradient(self, y_true, y_pred):

        raise NotImplementedError("Subclass must implement abstract method")

class MeanSquaredError(LossFunction):

    def loss(self, y_true, y_pred):

        return np.mean((y_true - y_pred)**2)

    def gradient(self, y_true, y_pred):

        return -2 * (y_true - y_pred) / y_true.shape[0]

class CrossEntropyLoss(LossFunction):

    def loss(self, y_true, y_pred):
        # Avoid division by zero or log of zero
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def gradient(self, y_true, y_pred):
        # Avoid division by zero
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return (y_pred - y_true) / (y_pred * (1 - y_pred)) / y_true.shape[0]

## Implementing optimization

*Gradient descent is used to update the model weights*


In [18]:
class Optimizer:

    def __init__(self, learning_rate):

        self.learning_rate = learning_rate
        # Use lists to store moments corresponding to parameters
        self.m = []  # First moment (mean)
        self.v = []  # Second moment (variance)
        self.beta1 = 0.9  # Exponential decay rate for the first moment estimates
        self.beta2 = 0.999 # Exponential decay rate for the second moment estimates
        self.epsilon = 1e-8 # Small constant to prevent division by zero
        self.t = 0 # Time step
        self._params_initialized = False # Flag to check if moments are initialized


    def update(self, params_and_grads):
        """
        Args:
            params_and_grads: A list of dictionaries, where each dictionary
                              contains 'params' (the parameter array) and
                              'grads' (the gradient array) for a layer.
        """
        self.t += 1

        if not self._params_initialized:
             # Initialize moments for each parameter if not already done
            for item in params_and_grads:
                params = item['params']
                self.m.append(np.zeros_like(params))
                self.v.append(np.zeros_like(params))
            self._params_initialized = True


        for i, item in enumerate(params_and_grads):
            params = item['params']
            grads = item['grads']

            # Update biased first and second moment estimates using list indexing
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grads
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grads**2)

            # Compute bias-corrected first and second moment estimates
            m_hat = self.m[i] / (1 - self.beta1**self.t)
            v_hat = self.v[i] / (1 - self.beta2**self.t)

            # Update parameters
            item['params'] -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)

## Training the network

The NN can be now trained to solve classification or regression problems

I have trained a NeuralNet on the *Wisconsin breast cancer dataset* which then outputs if the tumor is benign or malignant.


This is represented by the output of 1 if the tomor is benign and 0 otherwise.





I also have trained another instance on the California housing dataset, having the model predict the target variable, the house price.


In [19]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score


breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target.reshape(-1, 1)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Some preprocessing before feeding the data into the NN
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Network architecture params
input_size = X_train.shape[1]
output_size = 1
hidden_size = 32

binary_cancer_net = Network()
binary_cancer_net.add_layer(Dense(input_size, hidden_size))
binary_cancer_net.add_layer(ReLU())
binary_cancer_net.add_layer(Dense(hidden_size, output_size))
binary_cancer_net.add_layer(Sigmoid())


loss_function = CrossEntropyLoss()
optimizer = Optimizer(learning_rate=0.001)


# Training params
epochs = 10
batch_size = 32

binary_cancer_net.train(X_train, y_train, loss_function, optimizer, epochs, batch_size)

# Evaluate accuracy
def calculate_binary_accuracy(network, X, y_true_binary):
    y_pred = network.forward(X)

    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_true_binary, y_pred_binary)
    return accuracy

train_accuracy = calculate_binary_accuracy(binary_cancer_net, X_train, y_train)
test_accuracy = calculate_binary_accuracy(binary_cancer_net, X_test, y_test)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/10, Loss: 0.7183011349741765
Epoch 2/10, Loss: 0.6606205362795894
Epoch 3/10, Loss: 0.5549023955318061
Epoch 4/10, Loss: 0.42851060751544273
Epoch 5/10, Loss: 0.321767458799817
Epoch 6/10, Loss: 0.2626480949402606
Epoch 7/10, Loss: 0.21142753167379766
Epoch 8/10, Loss: 0.18130492224094905
Epoch 9/10, Loss: 0.16466441640852192
Epoch 10/10, Loss: 0.15349563468104513
Train Accuracy: 0.9692307692307692
Test Accuracy: 0.9736842105263158


In [39]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, mean_absolute_error


housing = fetch_california_housing()
X, y = housing.data, housing.target.reshape(-1, 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Some preprocessing before feeding the data into the NN
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Network architecture for regression
input_size = X_train.shape[1]
output_size = 1
hidden_size = 64

network = Network()
network.add_layer(Dense(input_size, hidden_size))
network.add_layer(ReLU())
network.add_layer(Dense(hidden_size, output_size))


loss_function = MeanSquaredError()
optimizer = Optimizer(learning_rate=0.001)

# Training
epochs = 15
batch_size = 8


network.train(X_train, y_train, loss_function, optimizer, epochs, batch_size)

# Make predictions on the test set
y_pred = network.forward(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")
print(f"Mean Absolute Error on Test Set: {mae}")

Epoch 1/15, Loss: 0.669293501963419
Epoch 2/15, Loss: 0.41949338893147536
Epoch 3/15, Loss: 0.39797497417166294
Epoch 4/15, Loss: 0.38579965617784356
Epoch 5/15, Loss: 0.37550733829900773
Epoch 6/15, Loss: 0.36599470717713933
Epoch 7/15, Loss: 0.36215694366250467
Epoch 8/15, Loss: 0.3569228348420627
Epoch 9/15, Loss: 0.3568232791435399
Epoch 10/15, Loss: 0.3546660956416732
Epoch 11/15, Loss: 0.3569506050072357
Epoch 12/15, Loss: 0.348592258471895
Epoch 13/15, Loss: 0.34649980532332153
Epoch 14/15, Loss: 0.34104329423008223
Epoch 15/15, Loss: 0.3378038142828495
Mean Squared Error on Test Set: 0.3444202377685985
Mean Absolute Error on Test Set: 0.40713583657078456


In [40]:
# Single data point
data_point = X_test[0:1] #



# Make a prediction using the trained network
prediction = network.forward(data_point)

predicted_value = prediction[0][0]
actual_value = y_test[0][0]

print(f"Predicted housing value: {predicted_value}")
print(f"Actual housing value: {actual_value}")

# Calculate percentage error
if actual_value != 0:
    percentage_error = ((actual_value - predicted_value) / actual_value) * 100
    print(f"Percentage Error: {percentage_error:.2f}%")
else:
    print("Cannot calculate percentage error as the actual value is zero.")

Predicted housing value: 0.6358440305750879
Actual housing value: 0.477
Percentage Error: -33.30%
